In [203]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [204]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,auc,roc_auc_score

In [205]:
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier,GradientBoostingClassifier
from sklearn.learning_curve import check_cv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC

In [392]:
train=pd.read_csv('train.csv')

In [393]:
test=pd.read_csv('test.csv')

In [394]:
train.head()

,ID,Gender,DOB,Lead_Creation_Date,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,...,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,Approved
0,APPC90493171225,Female,23/07/79,15/07/16,C10001,A,COM0044082,A,4.0,2000.0,...,N,S122,G,0.0,NaN,NaN,NaN,NaN,0,0
1,APPD40611263344,Male,07/12/86,04/07/16,C10003,A,COM0000002,C,1.0,3500.0,...,Y,S122,G,0.0,20000.0,2.0,13.25,953.0,10,0
2,APPE70289249423,Male,10/12/82,19/07/16,C10125,C,COM0005267,C,4.0,2250.0,...,Y,S143,B,0.0,45000.0,4.0,NaN,NaN,0,0
3,APPF80273865537,Male,30/01/89,09/07/16,C10477,C,COM0004143,A,4.0,3500.0,...,Y,S143,B,0.0,92000.0,5.0,NaN,NaN,7,0
4,APPG60994436641,Male,19/04/85,20/07/16,C10002,A,COM0001781,A,4.0,10000.0,...,Y,S134,B,2500.0,50000.0,2.0,NaN,NaN,10,0


In [395]:
y=train['Approved']
train.drop(['Approved'],axis=1,inplace=True)

In [396]:
train.shape

(69713, 21)

In [397]:
train['Employer_Category1'].unique()

array(['A', 'C', 'B', nan], dtype=object)

In [398]:
train['Employer_Category2'].unique()

array([  4.,   1.,   3.,   2.,  nan])

In [399]:
test.shape

(30037, 21)

In [400]:
from sklearn.feature_selection import SelectPercentile,chi2

In [401]:
train.drop(['ID'],axis=1,inplace=True)

In [402]:
ID=test['ID']
test.drop(['ID'],axis=1,inplace=True)

In [411]:
le=LabelEncoder()
features=['Gender','City_Code','City_Category','Employer_Category1','Contacted','Source_Category','Primary_Bank_Type']
for feature in features:
    train[feature]=le.fit_transform(train[feature])
    test[feature]=le.fit_transform(test[feature])

In [415]:
train.isnull().sum()

Gender                0
City_Code             0
City_Category         0
Employer_Category1    0
Employer_Category2    0
Monthly_Income        0
Primary_Bank_Type     0
Contacted             0
Source_Category       0
Existing_EMI          0
Loan_Amount           0
Loan_Period           0
Interest_Rate         0
EMI                   0
Var1                  0
dtype: int64

In [403]:
train.drop(['DOB','Lead_Creation_Date','Employer_Code','Customer_Existing_Primary_Bank_Code','Source'],axis=1,inplace=True)

In [404]:
train.shape

(69713, 15)

In [405]:
test.drop(['DOB','Lead_Creation_Date','Employer_Code','Customer_Existing_Primary_Bank_Code','Source'],axis=1,inplace=True)

In [406]:
test.shape

(30037, 15)

In [412]:
train['Primary_Bank_Type'].fillna(train['Primary_Bank_Type'].mode(),inplace=True)
train['City_Code'].fillna(train['City_Code'].mode(),inplace=True)
train['City_Category'].fillna(train['City_Category'].mode(),inplace=True)
train['Employer_Category1'].fillna(train['Employer_Category1'].mode(),inplace=True)
train['Employer_Category2'].fillna(train['Employer_Category2'].median(),inplace=True)
train['Existing_EMI'].fillna(train['Existing_EMI'].mean(),inplace=True)
train['Loan_Amount'].fillna(train['Loan_Amount'].mean(),inplace=True) 
train['Loan_Period'].fillna(train['Loan_Period'].mean(),inplace=True)
train['Interest_Rate'].fillna(train['Interest_Rate'].mean(),inplace=True)
train['EMI'].fillna(train['EMI'].mean(),inplace=True)

In [414]:
test.isnull().sum()

Gender                0
City_Code             0
City_Category         0
Employer_Category1    0
Employer_Category2    0
Monthly_Income        0
Primary_Bank_Type     0
Contacted             0
Source_Category       0
Existing_EMI          0
Loan_Amount           0
Loan_Period           0
Interest_Rate         0
EMI                   0
Var1                  0
dtype: int64

In [413]:
test['Primary_Bank_Type'].fillna(test['Primary_Bank_Type'].mode(),inplace=True)
test['City_Code'].fillna(test['City_Code'].mode(),inplace=True)
test['City_Category'].fillna(test['City_Category'].mode(),inplace=True)
test['Employer_Category1'].fillna(test['Employer_Category1'].mode(),inplace=True)
test['Employer_Category2'].fillna(test['Employer_Category2'].median(),inplace=True)
test['Existing_EMI'].fillna(test['Existing_EMI'].mean(),inplace=True)
test['Loan_Amount'].fillna(test['Loan_Amount'].mean(),inplace=True) 
test['Loan_Period'].fillna(test['Loan_Period'].mean(),inplace=True)
test['Interest_Rate'].fillna(test['Interest_Rate'].mean(),inplace=True)
test['EMI'].fillna(test['EMI'].mean(),inplace=True)

In [417]:
percent=SelectPercentile(score_func=chi2,percentile=50)

In [418]:
X=train
X.head()

,Gender,City_Code,City_Category,Employer_Category1,Employer_Category2,Monthly_Income,Primary_Bank_Type,Contacted,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1
0,0,1,1,1,4.0,2000.0,2,0,6,0.0,39429.982859,3.890629,19.21357,1101.466242,0
1,1,3,1,3,1.0,3500.0,2,1,6,0.0,20000.000000,2.000000,13.25000,953.000000,10
2,1,125,3,3,4.0,2250.0,1,1,1,0.0,45000.000000,4.000000,19.21357,1101.466242,0
3,1,477,3,1,4.0,3500.0,1,1,1,0.0,92000.000000,5.000000,19.21357,1101.466242,7
4,1,2,1,1,4.0,10000.0,2,1,1,2500.0,50000.000000,2.000000,19.21357,1101.466242,10


In [419]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

In [420]:
X_train.shape,X_test.shape

((48799, 15), (20914, 15))

In [421]:
#LogisticRegression
clf=LogisticRegression()
clf.fit(X_train,y_train)
preds=clf.predict(X_test)
print 'accuracy:',accuracy_score(preds,y_test)
print 'roc auc:',roc_auc_score(preds,y_test)

accuracy: 0.986468394377
roc auc: 0.576685160385


In [422]:
C_param_range = [0.001,0.01,0.1,1,10,100,1e-6]
for i  in C_param_range:
    lr=LogisticRegression(penalty='l2',C=i,n_jobs=4,random_state=42)
    lr.fit(X_train,y_train)
    preds=lr.predict(X_test)
    print 'accuracy:',accuracy_score(preds,y_test)
    print 'roc auc:',roc_auc_score(preds,y_test)

accuracy: 0.986468394377
roc auc: 0.576685160385
accuracy: 0.986468394377
roc auc: 0.576685160385
accuracy: 0.986468394377
roc auc: 0.576685160385
accuracy: 0.986468394377
roc auc: 0.576685160385
accuracy: 0.986468394377
roc auc: 0.576685160385
accuracy: 0.986468394377
roc auc: 0.576685160385
accuracy: 0.986516209238
roc auc: 0.593352145009


In [423]:
#ExtraTreesClassifier
clf=ExtraTreesClassifier()
clf.fit(X_train,y_train)
preds=clf.predict(X_test)
print 'accuracy:',accuracy_score(preds,y_test)
print 'roc auc:',roc_auc_score(preds,y_test)

accuracy: 0.984125466195
roc auc: 0.50242711888


In [424]:
#RandomForestClassifier
clf=RandomForestClassifier()
clf.fit(X_train,y_train)
preds=clf.predict(X_test)
print 'accuracy:',accuracy_score(preds,y_test)
print 'roc auc:',roc_auc_score(preds,y_test)

accuracy: 0.985894616047
roc auc: 0.493324720069


In [425]:
#GradientBoostingClassifier
clf=GradientBoostingClassifier()
clf.fit(X_train,y_train)
preds=clf.predict(X_test)
print 'accuracy:',accuracy_score(preds,y_test)
print 'roc auc:',roc_auc_score(preds,y_test)

accuracy: 0.986277134934
roc auc: 0.493327274467


In [426]:
#DecisionTreeClassifier
clf=DecisionTreeClassifier()
clf.fit(X_train,y_train)
preds=clf.predict(X_test)
print 'accuracy:',accuracy_score(preds,y_test)
print 'roc auc:',roc_auc_score(preds,y_test)

accuracy: 0.969111599885
roc auc: 0.51364018137


In [248]:
#SVC
clf=SVC()
clf.fit(X_train,y_train)
preds=clf.predict(X_test)
print 'accuracy:',accuracy_score(preds,y_test)
print 'roc auc:',roc_auc_score(preds,y_test)

accuracy: 0.98661183896
roc auc: 0.493329507962


In [427]:
X=train
train.head()

,Gender,City_Code,City_Category,Employer_Category1,Employer_Category2,Monthly_Income,Primary_Bank_Type,Contacted,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1
0,0,1,1,1,4.0,2000.0,2,0,6,0.0,39429.982859,3.890629,19.21357,1101.466242,0
1,1,3,1,3,1.0,3500.0,2,1,6,0.0,20000.000000,2.000000,13.25000,953.000000,10
2,1,125,3,3,4.0,2250.0,1,1,1,0.0,45000.000000,4.000000,19.21357,1101.466242,0
3,1,477,3,1,4.0,3500.0,1,1,1,0.0,92000.000000,5.000000,19.21357,1101.466242,7
4,1,2,1,1,4.0,10000.0,2,1,1,2500.0,50000.000000,2.000000,19.21357,1101.466242,10


In [428]:
test.head()

,Gender,City_Code,City_Category,Employer_Category1,Employer_Category2,Monthly_Income,Primary_Bank_Type,Contacted,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1
0,1,28,3,1,4.0,2150.0,2,1,1,0.0,10000.000000,3.000000,20.000000,372.000000,4
1,1,3,1,3,4.0,4200.0,2,1,2,0.0,69000.000000,5.000000,24.000000,1985.000000,7
2,0,9,2,2,4.0,1000.0,2,0,1,0.0,39482.990201,3.903116,19.280537,1094.914836,0
3,0,5,1,1,3.0,1465.0,2,0,2,0.0,39482.990201,3.903116,19.280537,1094.914836,0
4,1,5,1,1,4.0,2340.0,2,1,1,500.0,10000.000000,2.000000,19.280537,1094.914836,0


In [258]:
lr=LogisticRegression(penalty='l2',C=1e-6,n_jobs=4,random_state=42)
lr.fit(X_train,y_train)
preds=lr.predict(test)
sub=pd.DataFrame()
sub['ID']=ID
sub['Approved']=pd.Series(preds)
sub.head()

,ID,Approved
0,APPA70109647212,0
1,APPB10687939341,0
2,APPC80449411414,0
3,APPD30665094501,0
4,APPE80379821637,0


In [259]:
sub.to_csv('submission1.csv',index=False)

In [440]:
from sklearn.feature_selection import  SelectKBest,chi2


In [357]:
X_new=SelectKBest(k=6,score_func=chi2).fit_transform(X,y)

In [358]:
def train_score(X,y):
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

    print 'LogisticRegression'
    lr=LogisticRegression(n_jobs=-1,penalty='l2')
    lr.fit(X_train,y_train)
    preds=lr.predict(X_test)
    print 'accuracy:',accuracy_score(preds,y_test)
    print 'roc auc:',roc_auc_score(preds,y_test)

    print 'ExtraTreesClassifier'
    et=ExtraTreesClassifier(max_features=5,n_estimators=15,n_jobs=-1)
    et.fit(X_train,y_train)
    preds=et.predict(X_test)
    print 'accuracy:',accuracy_score(preds,y_test)
    print 'roc auc:',roc_auc_score(preds,y_test)

    print 'RandomForestClassifier'
    rf=RandomForestClassifier(n_estimators=16,n_jobs=-1)
    rf.fit(X_train,y_train)
    preds=rf.predict(X_test)
    print 'accuracy:',accuracy_score(preds,y_test)
    print 'roc auc:',roc_auc_score(preds,y_test)

    print 'GradientBoostingClassifier'
    gb=GradientBoostingClassifier(n_estimators=15)
    gb.fit(X_train,y_train)
    preds=gb.predict(X_test)
    print 'accuracy:',accuracy_score(preds,y_test)
    print 'roc auc:',roc_auc_score(preds,y_test)

    print 'DecisionTreeClassifier'
    dt=DecisionTreeClassifier()
    dt.fit(X_train,y_train)
    preds=dt.predict(X_test)
    print 'accuracy:',accuracy_score(preds,y_test)
    print 'roc auc:',roc_auc_score(preds,y_test)
    
    return (lr,et,rf,gb,dt)

In [429]:
(lr,et,rf,gb,dt)=train_score(X,y)

LogisticRegression
accuracy: 0.986468394377
roc auc: 0.576685160385
ExtraTreesClassifier
accuracy: 0.983647317586
roc auc: 0.508281731363
RandomForestClassifier
accuracy: 0.986372764655
roc auc: 0.555851191046
GradientBoostingClassifier
accuracy: 0.986468394377
roc auc: 0.493328550933
DecisionTreeClassifier
accuracy: 0.968681266138
roc auc: 0.517720772008


In [434]:
dt.fit(X_train,y_train)
preds=dt.predict(test)
sub=pd.DataFrame()
sub['ID']=ID
sub['Approved']=pd.Series(preds)


In [435]:
sub.to_csv('submission1.csv',index=False)

In [431]:
from xgboost import XGBClassifier

In [443]:
xgb=XGBClassifier()

bst = xgb.fit(X_train,y_train)
# make prediction
preds = xgb.predict(X_test)
print accuracy_score(preds,y_test)
print preds
#print roc_auc_score(preds,y_test)

0.98665965382
[0 0 0 ..., 0 0 0]


In [437]:
preds=pd.Series(preds)
preds.unique()

array([0])

In [444]:
preds=xgb.predict(test)
sub=pd.DataFrame()
sub['ID']=ID
sub['Approved']=pd.Series(preds)
sub.to_csv('submission1.csv',index=False)

In [441]:
X_new=SelectKBest(k=6,score_func=chi2).fit_transform(X,y)

In [442]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)